This file will be used to contain data processing components

In [1]:
%pip install boto3 pyspark delta-spark python-dotenv

  Using cached pyspark-3.5.1-py2.py3-none-any.whl
  Obtaining dependency information for delta-spark from https://files.pythonhosted.org/packages/0a/ee/dcf357307bdf8ef5436bb147159cf42c339e556b09c2eb1aec7e7b472022/delta_spark-3.1.0-py3-none-any.whl.metadata
  Using cached delta_spark-3.1.0-py3-none-any.whl.metadata (1.9 kB)
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
  Obtaining dependency information for importlib-metadata>=1.0.0 from https://files.pythonhosted.org/packages/2d/0a/6


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: d:\Data_major\icttm_submit\env\Scripts\python.exe -m pip install --upgrade pip


In [2]:
import os
from pyspark.sql import SparkSession

In [3]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'i8UnD6H3PyxR0UFEuUY7')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', '8JFkDk4otgUQd1EOlKIfQIQPq8EzxqEBSzBz8dda')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

#### First methods: using spark to connect minio and load json data into spark dataframes, then merge them and write into single one json file

I change to use hadoop-aws:3.3.4 beacuse in the folder `env\Lib\site-packages\pyspark\jars`, the hadoop files have version 3.3.4

In [7]:
# You need to more configuration if you want to use minio as object storage 
# (hint: maybe you can using .config() method to set the configuration if you want using spark to read/write data from/to minio)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config('spark.hadoop.fs.s3a.path.style.access', "true") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config('spark.hadoop.fs.s3a.access.key', obj_storage_access_key) \
    .config('spark.hadoop.fs.s3a.secret.key', obj_storage_secret_key) \
    .config('spark.hadoop.fs.s3a.endpoint', obj_storage_endpoint) \
    .getOrCreate()

# Define path to read data from minio
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

df1 = spark.read.json(path_1)
df2 = spark.read.json(path_2)
df3 = spark.read.json(path_3)

df1.show(truncate=False)
df2.show(truncate=False)
df3.show(truncate=False)

df_merged = df1.union(df2).union(df3)
df_merged.show(truncate=False)

df_merged = df_merged.repartition(1)
df_merged.write.format("json").mode("overwrite").save("merge_data")

+------------------------+----+----+------------+
|_corrupt_record         |id  |name|type        |
+------------------------+----+----+------------+
|[                       |null|null|null        |
|    {                   |null|null|null        |
|        "id": "0004",   |null|null|null        |
|        "type": "donut",|null|null|null        |
|        "name": "Jelly",|null|null|null        |
|        "ppu": 0.65,    |null|null|null        |
|        "batters": {    |null|null|null        |
|            "batter": [ |null|null|null        |
|null                    |1001|null|Regular     |
|null                    |1002|null|Chocolate   |
|null                    |1003|null|Blueberry   |
|null                    |1004|null|Devil's Food|
|            ]           |null|null|null        |
|        },              |null|null|null        |
|        "topping": [    |null|null|null        |
|null                    |5001|null|None        |
|null                    |5002|null|Glazed      |


#### Second methods: Using minio and json libraries to get content of json files first and then merge and load data into a spark dataframe

Since the result of first method is messy and hard for analyzing or cleaning data, I have to find another solution and come up with this second method

In [5]:
%pip install minio

  Obtaining dependency information for minio from https://files.pythonhosted.org/packages/45/a4/6f278051ad2bc03f3a0fdb4e182c9529009b0357631c2bb7c6ae70b4b0f6/minio-7.2.5-py3-none-any.whl.metadata
  Using cached minio-7.2.5-py3-none-any.whl.metadata (6.4 kB)
  Obtaining dependency information for argon2-cffi from https://files.pythonhosted.org/packages/a4/6a/e8a041599e78b6b3752da48000b14c8d1e8a04ded09c88c714ba047f34f5/argon2_cffi-23.1.0-py3-none-any.whl.metadata
  Using cached argon2_cffi-23.1.0-py3-none-any.whl.metadata (5.2 kB)
  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/1f/90/d131c0eb643290230dfa4108b7c2d135122d88b714ad241d77beb4782a76/pycryptodome-3.20.0-cp35-abi3-win_amd64.whl.metadata
  Using cached pycryptodome-3.20.0-cp35-abi3-win_amd64.whl.metadata (3.4 kB)
  Obtaining dependency information for argon2-cffi-bindings from https://files.pythonhosted.org/packages/37/2c/e34e47c7dee97ba6f01a6203e0383e15b60fb85d78ac9a15cd066f6fe28b/


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: d:\Data_major\icttm_submit\env\Scripts\python.exe -m pip install --upgrade pip


In [6]:
# Merge data using Minio and write data to result.json file
import json
from minio import Minio

bucket_name = "data"
obj_storage_endpoint_without_path = obj_storage_endpoint.split('/')[2]

#initialize minio client
minio_client = Minio(obj_storage_endpoint_without_path, access_key=obj_storage_access_key, secret_key=obj_storage_secret_key, secure=False)

# Get data from minio
data1 = minio_client.get_object(bucket_name, "data-raw/data.json").read().decode('utf-8')
data2 = minio_client.get_object(bucket_name, "data-raw/data2.json").read().decode('utf-8')
data3 = minio_client.get_object(bucket_name, "data-raw/data3.json").read().decode('utf-8')

json_data1 = json.loads(data1)
json_data2 = json.loads(data2)
json_data3 = json.loads(data3)

# Merge data
merged_data = json_data1 + json_data2 + json_data3
with open('result.json', 'w') as f:
    json.dump(merged_data, f)

In [8]:
# Upload file to minio
import boto3
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)

upload_file_to_minio('result.json', 'data', 'data-result/result.json')


In [9]:
# Load result.json file into spark dataframe
result_df = spark.read.json(spark.sparkContext.parallelize(merged_data))
row_count = result_df.count()
print("Row count: ", row_count)
result_df.printSchema()
result_df.show(truncate=False)

Row count:  19
root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- filling: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

+-------------------------------------------------------------------------------+--------------------------------------------------------------------------------+----+-------------------+----+------------------------------------------------------------

#### Check duplicate row

In [10]:
from pyspark.sql import functions as F

# Group by all columns and count occurrences of each group
duplicate_rows = result_df.groupBy(result_df.columns).count().where(F.col('count') > 1)

if duplicate_rows.count() == 0:
    print("No duplicate rows found.")
else:
    print("Duplicate rows found:")
    duplicate_rows.show(truncate=False)

Duplicate rows found:
+-------------------------------------------------------------------------------+-------+----+-----+----+-------------------------------------------------------------------------------+-----+-----+
|batters                                                                        |filling|id  |name |ppu |topping                                                                        |type |count|
+-------------------------------------------------------------------------------+-------+----+-----+----+-------------------------------------------------------------------------------+-----+-----+
|{[{1001, Regular}, {1002, Chocolate}, {1003, Blueberry}, {1004, Devil's Food}]}|null   |0004|Jelly|0.65|[{5001, None}, {5002, Glazed}, {5003, Chocolate}, {5004, Maple}, {5005, Sugar}]|donut|2    |
+-------------------------------------------------------------------------------+-------+----+-----+----+-------------------------------------------------------------------------------+-

#### Remove duplicate rows

In [11]:
# Remove duplicate rows
result_df_no_duplicates = result_df.dropDuplicates()
row_count_no_duplicates = result_df_no_duplicates.count()
print("Row count after removing duplicates: ", row_count_no_duplicates)
result_df_no_duplicates.show(truncate=False)

Row count after removing duplicates:  18
+-------------------------------------------------------------------------------+--------------------------------------------------------------------------------+----+-------------------+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|batters                                                                        |filling                                                                         |id  |name               |ppu |topping                                                                                                                                                                             |type |
+-------------------------------------------------------------------------------+--------------------------------------------------------------------------------+----+-------------------+----+-------

### Save final result into a csv file
##### About the final requirement, I'm confused at this line: `The final result is the csv file uploaded to minio or not and can the csv file be read or not.` Because from the begining, I process with json files and upload the json file (result.json) to minio but not any csv files.

In [12]:
# This code will case error because batters, filling, topping columns has data type which is not supported by csv
# result_df_no_duplicates.write.csv("final_result.csv", header=True)

# So I have to convert the data type of those columns to string
from pyspark.sql.types import StringType

result_df_no_duplicates = result_df_no_duplicates.withColumn("batters", result_df_no_duplicates["batters"].cast(StringType()))
result_df_no_duplicates = result_df_no_duplicates.withColumn("filling", result_df_no_duplicates["filling"].cast(StringType()))
result_df_no_duplicates = result_df_no_duplicates.withColumn("topping", result_df_no_duplicates["topping"].cast(StringType()))

# Create a folder named final_result and inside the folder there will be one csv file
result_df_no_duplicates.write.mode("overwrite").csv("final_result", header=True)

#### Check if the csv can be read or not

In [13]:
verified_csv_df = spark.read.csv("final_result\part-00000-a0c6a582-4876-4b7d-8a7b-68085592f674-c000.csv", header=True)
count_row = verified_csv_df.count()
print("Row count: ", count_row)
verified_csv_df.printSchema()
verified_csv_df.show(truncate=False)

Row count:  18
root
 |-- batters: string (nullable = true)
 |-- filling: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: string (nullable = true)
 |-- topping: string (nullable = true)
 |-- type: string (nullable = true)

+-------------------------------------------------------------------------------+--------------------------------------------------------------------------------+----+-------------------+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|batters                                                                        |filling                                                                         |id  |name               |ppu |topping                                                                                                                                              